In [0]:

from aadatapipelinecore.core.log import logger
import pyspark.sql.functions as F
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf


source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)
print path_list[96]



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/

In [0]:

from aadatapipelinecore.core.log import logger
import pyspark.sql.functions as F
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf


source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)

    
    
    
def get_monthly_sample_data(monthly):
    
    review_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month={}/".format(monthly))
    review_df.registerTempTable("reviews")
    ios_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'apple-store' LIMIT 50").cache()
    ios_samples_df.count()
    android_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'google-play' LIMIT 50").cache()
    android_samples_df.count()
    samples = []
    month_index = monthly.replace("-","")
    for idx, df in [
        ("int-ss-review_v1-apple-store-ios-all-{}".format(month_index), ios_samples_df),
        ("int-ss-review_v1-google-play-android-all-{}".format(month_index), android_samples_df)]:
        print df.columns
        rows = (
            df
            .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
            .drop("time")
            .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
            .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
            .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
            .withColumn("english_content", F.when(df.content_language == "en", df.content))
            .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
            .withColumn("english_title", F.when(df.title_language == "en", df.title))
            .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
            .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
            .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
            .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
            .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
            .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
            .withColumn("other_language_content", F.when(
                (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
                (df.content_language != "ja") & (df.content_language != "ko"), df.content))
            .withColumn("other_language_reply", F.when(
                (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
                (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
            .withColumn("other_language_title", F.when(
                (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
                (df.title_language != "ja") & (df.title_language != "ko"), df.title))
            .collect()
        )
        samples.append((idx, rows))
    return samples
    
    
sample_data_set=[]
for x in path_list[100:]:
    sample_data_set.append(get_monthly_sample_data(x.split("/")[3].split("=")[1]))
    
def compare(doc, row):
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
    row_dict =  row.asDict()
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source
    
    
for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)


In [0]:

import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

print sample_data_set[10]
# def compare(doc, row):
#     keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
#     row_dict =  row.asDict()
#     for key in keys:
#         if key in ("app_id", "_identifier"):
#             assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
#         else:
#             assert doc[key] == row_dict[key], "{} is diffrent".format(key)
# def es_doc(index, doc_id, routing_id):
#     headers = {
#         'content-type': 'application/json'
#     }
#     auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
#     url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
#     resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
#     doc = json.loads(resp.text)
#     source = doc['_source']
#     source['review_id'] = doc["_id"]
#     return source
# for idx, rows in samples:
#     for row in rows:
#         doc = es_doc(idx, row.review_id, row.app_id)
#         compare(doc, row)




In [0]:


import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get
def compare(doc, row):
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
    row_dict =  row.asDict()
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source
for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)


